
# FAQ RAG — Notebook Único (FAISS + MiniLM + FLAN‑T5)

1) **Ingestão** de `.txt` locais  
2) **Embeddings** com `all-MiniLM-L6-v2`  
3) **Indexação** com FAISS  
4) **Recuperação** top‑k  
5) **Geração** com `flan-t5-base` (100% local) 
6) **UI opcional** com Gradio dentro do notebook  

> Requisitos: `faiss-cpu`, `sentence-transformers`, `transformers`, `accelerate`, `torch`, `pandas`, `gradio` (opcional).


# OpenAi

In [3]:
import configparser
import openai

# Ler config.ini
config = configparser.ConfigParser()
config.read("config.ini")

# Pegar a chave
openai.api_key = config["openai"]["api_key"]


In [ ]:
from pathlib import Path
import re, configparser
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter

# ========== 1) Ler chave do config.ini ==========
config = configparser.ConfigParser()
config.read("config.ini")
OPENAI_API_KEY = config["openai"]["api_key"]

# ====== 2) Split seção-ciente + sub-split por tamanho ======
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

ARQUIVO = Path("guia_equipamentos_snowboard.txt")
texto = ARQUIVO.read_text(encoding="utf-8")

# 2.1 primeiro quebrar por seção "###"
secoes = [s.strip() for s in re.split(r"\n(?=### )", texto) if s.strip()]

# 2.2 sub-split para seções longas (preserva o título dentro de cada pedaço)
subsplitter = RecursiveCharacterTextSplitter(
    chunk_size=300,      
    chunk_overlap= 50,
    separators=["\n\n", "\n", ". ", " "]
)

docs = []
for sec in secoes:
    # se a seção é curta, fica inteira; se for longa, quebramos com overlap
    if len(sec) <= 700:
        docs.append(Document(page_content=sec, metadata={"source": ARQUIVO.name}))
    else:
        parts = subsplitter.split_text(sec)
        for p in parts:
            # garante que o título "### ..." permaneça no pedaço
            if not p.lstrip().startswith("###"):
                # pega o cabeçalho da seção original
                header = sec.splitlines()[0]
                p = header + "\n" + p
            docs.append(Document(page_content=p.strip(), metadata={"source": ARQUIVO.name}))

print(f"{len(docs)} chunks criados (seção‑cientes)")

# ====== 3) Embeddings + FAISS (modelo multilíngue melhor p/ PT-BR) ======
from langchain_community.embeddings import HuggingFaceEmbeddings

emb = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
vectorstore = FAISS.from_documents(docs, emb)

# ====== 4) LLM OpenAI (gpt-4o-mini) ======
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-4o-mini", temperature=0.3)

# ====== 5) Prompt enxuto ======
from langchain.prompts import PromptTemplate
PROMPT = PromptTemplate.from_template("""\
Você é um assistente em português.
Responda à pergunta de forma **curta (2–4 frases)**, clara e natural, usando **SOMENTE** o contexto.
Não copie frases do contexto; reescreva com suas palavras.
Se a resposta não estiver no contexto, diga: "Não encontrei essa informação nos documentos."

[PERGUNTA]
{question}

[CONTEXTO]
{context}

[RESPOSTA]
""")

# ====== 6) RetrievalQA (MMR melhora foco) ======
from langchain.chains import RetrievalQA

retriever = vectorstore.as_retriever(
    search_type="mmr",           # em vez de pure similarity
    search_kwargs={"k": 5, "fetch_k": 13, "lambda_mult": 0.5}
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": PROMPT},
    return_source_documents=True,
)


114 chunks criados (seção‑cientes)


C:\Users\berna\AppData\Local\Temp\ipykernel_14456\3207146501.py:53: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")


In [ ]:

# ====== 7) Teste ======
pergunta = "Como escolher o tamanho ideial para a prancha de snowboard?"
resp = qa.invoke({"query": pergunta})

print("\nPergunta:", pergunta)
print("\nResposta:\n", resp["result"])
print("\nFontes:")
for d in resp["source_documents"]:
    print("-", d.metadata.get("source"))


In [6]:
# --- CHAT (ipywidgets) com dedupe + quebra de linha ---
from ipywidgets import Text, Textarea, Button, Output, VBox, HBox, IntSlider, Checkbox
from IPython.display import display
from textwrap import shorten

try:
    qa  # noqa
except NameError:
    raise RuntimeError("Objeto `qa` não encontrado. Crie o RetrievalQA antes do widget.")

inp = Text(placeholder="Digite sua pergunta e tecle Enter", layout={'width': '100%'})
send_btn = Button(description="Enviar")
clear_btn = Button(description="Limpar", button_style="warning")
k_slider = IntSlider(value=1, min=1, max=5, step=1, description='k:', continuous_update=False)
show_sources = Checkbox(value=True, description='Mostrar fontes')

# Transcript como Textarea (quebra linha + scroll)
log = Textarea(
    value="RAG Chat (ipywidgets) pronto! Ajuste k se quiser, pergunte algo como: 'Quais os tipos de shape de prancha de snowboard?'\n",
    layout={'width': '100%', 'height': '360px'},
    disabled=True
)

# aplica k inicial
try:
    qa.retriever.search_kwargs.update({"k": k_slider.value})
except Exception:
    pass

def _set_k(change=None):
    try:
        qa.retriever.search_kwargs.update({"k": k_slider.value})
    except Exception as e:
        _append(f"[Aviso] Não consegui ajustar k: {e}")

k_slider.observe(_set_k, names='value')

def _append(text: str):
    # adiciona linha e força scroll ao final
    log.value = (log.value + ("" if log.value.endswith("\n") else "\n") + text).rstrip() + "\n"

busy = False  # trava anti-duplicidade

def _ask(q: str):
    global busy
    q = q.strip()
    if not q or busy:
        return
    busy = True
    try:
        _append(f"Você: {q}")
        resp = qa.invoke({"query": q})
        answer = (resp.get("result") or "").strip()
        _append(f"Assistente: {answer}\n")
        if show_sources.value:
            fontes = resp.get("source_documents", []) or []
            if fontes:
                _append("Fontes:")
                for d in fontes:
                    src = d.metadata.get("source", "desconhecida")
                    snippet = shorten((d.page_content or "").replace("\n", " "), width=140, placeholder="...")
                    _append(f" - {src}: {snippet}")
                _append("")  # linha em branco
    except Exception as e:
        _append(f"[Erro] {e}")
    finally:
        busy = False

def _submit(_):
    q = inp.value
    inp.value = ""
    _ask(q)

def _click(_):
    _submit(None)

def _clear(_):
    log.value = ""

inp.on_submit(_submit)
send_btn.on_click(_click)
clear_btn.on_click(_clear)

ui = VBox([
    HBox([inp, send_btn, clear_btn]),
    HBox([k_slider, show_sources]),
    log
])
display(ui)


C:\Users\berna\AppData\Local\Temp\ipykernel_14456\2327325990.py:80: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  inp.on_submit(_submit)
